**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part III: LLM

Please see the description of the assignment in the README file (section 3) <br>
**Guide notebook**: [guides/llm_guide.ipynb](guides/llm_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: how do they compare with the results from Part I, BoW?, and part II, BERT? Are there any hyperparameters or prompting techniques that are particularly important?

* You should follow the steps given in the `llm_guide` notebook

<br>


***

In [8]:
# imports for the project

import pandas as pd

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [9]:

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

In [10]:
label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac = 1e-2, label_map = label_map, seed=42) -> pd.DataFrame:
    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

# train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
# del train
del test

test_df.shape, # train_df.shape, 

((760, 2),)

In [11]:
from decouple import Config, RepositoryEnv
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

In [12]:
# Load the environment variables using python-decouple
# The .env file should be in the root of the project
# The .env file should NOT be committed to the repository

config = Config(RepositoryEnv(".env.myapikey"))

# Load the credentials
WX_API_KEY = config("WX_API_KEY")

In [13]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)

client = APIClient(
    credentials=credentials, 
    project_id="8b2fe967-dfcc-4e5c-ae77-ac79a96ed699"
)


In [ ]:
# Unfortunately, the connection to the ML Service seems to be broken which is blocking me from running the code below. To be validated once the connection is back up.

model = ModelInference(
    api_client=client,
    model_id="ibm/granite-13b-instruct-v2",
)

Failure during Get available foundation models. (GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-02-19&project_id=8b2fe967-dfcc-4e5c-ae77-ac79a96ed699&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"project_id 8b2fe967-dfcc-4e5c-ae77-ac79a96ed699 is not associated with a WML instance","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"cd1144cc6543d2fa69ad742531f794cc","status_code":403}
Unable to get model specifications from url: https://us-south.ml.cloud.ibm.com
Reason: Failure during Get available foundation models. (GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-02-19&project_id=8b2fe967-dfcc-4e5c-ae77-ac79a96ed699&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"proje

WMLClientError: Unable to get model specifications from url: https://us-south.ml.cloud.ibm.com
Reason: Failure during Get available foundation models. (GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-02-19&project_id=8b2fe967-dfcc-4e5c-ae77-ac79a96ed699&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200)
Status code: 403, body: {"errors":[{"code":"no_associated_service_instance_error","message":"project_id 8b2fe967-dfcc-4e5c-ae77-ac79a96ed699 is not associated with a WML instance","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"cd1144cc6543d2fa69ad742531f794cc","status_code":403}

In [ ]:
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
import pandas as pd
from sklearn.metrics import classification_report 
from tqdm import tqdm

In [ ]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

In [ ]:
label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac : float = 1e-2, label_map : dict[int, str] = label_map, seed : int = 42) -> pd.DataFrame:
    """ Preprocess the dataset 

    Operations:
    - Map the label to the corresponding category
    - Filter out the labels not in the label_map
    - Sample a fraction of the dataset (stratified by label)

    Args:
    - df (pd.DataFrame): The dataset to preprocess
    - frac (float): The fraction of the dataset to sample in each category
    - label_map (dict): A mapping of the original label to the new label
    - seed (int): The random seed for reproducibility

    Returns:
    - pd.DataFrame: The preprocessed dataset
    """

    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')[["text", "label"]]
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

# train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
# del train
del test

test_df.shape # , train_df.shape

In [ ]:
PARAMS = TextGenParameters(
    temperature=0,              # Higher temperature means more randomness - In this case we don't want randomness
    max_new_tokens=10,          # Maximum number of tokens to generate
    stop_sequences=[".", "\n"], # Stop generating text when these sequences are encountered
)

model = ModelInference(
    api_client=client,
    model_id="ibm/granite-13b-instruct-v2",  # We could also try a larger model!
    params=PARAMS
)

In [ ]:
SYSTEM_PROMPT = """You task is to classify news stories into one of five categories

CATEGORIES:
{categories}

TEXT:
{text}

Please assign the correct category to the text. Answer with the correct category and nothing else.

Category:
"""

In [ ]:
CATEGORIES = "- " + "\n- ".join(test_df["label"].unique())  # Create a string with all categories

predictions = []

for text in tqdm(test_df["text"]):

    # format the prompt with the categories and the text
    prompt = SYSTEM_PROMPT.format(categories=CATEGORIES, text=text)
    
    # generate the response from the model
    response = model.generate(prompt)

    # extract the generated text from the response
    prediction = response["results"][0]["generated_text"].strip()

    # append the prediction to the list of predictions
    predictions.append(prediction)

In [ ]:
print(classification_report(test_df.label, predictions))